In [21]:
import pandas as pd


In [22]:
scripts = pd.read_csv('data/lotr_scripts.csv')

In [23]:
scripts = scripts.drop(columns=['Unnamed: 0'])

In [24]:
scripts['movie'].value_counts()

The Two Towers                 1010
The Return of the King          873
The Fellowship of the Ring      507
Name: movie, dtype: int64

In [25]:


# Filter the DataFrame for the specified movie
filtered_scripts = scripts[scripts['movie'] == 'The Fellowship of the Ring']

In [26]:
filtered_scripts

,char,dialog,movie
